In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
conll = datasets.load_dataset("conll2003")
CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names
print(CONLL_NER_TAGS)
conll["test"][2]

Found cached dataset conll2003 (/Users/azatsultanov/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


{'id': '2',
 'tokens': ['AL-AIN', ',', 'United', 'Arab', 'Emirates', '1996-12-06'],
 'pos_tags': [22, 6, 22, 22, 23, 11],
 'chunk_tags': [11, 0, 11, 12, 12, 12],
 'ner_tags': [5, 0, 5, 6, 6, 0]}

In [3]:
from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)

# Load pretrained model and tokenizer for English NER task (dslim/bert-base-NER)
model_name = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

In [4]:
import numpy as np
import torch

class ModelWrapper():
    def __init__(self, model, tokenizer):
        self.model = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy=None)

    def _unite_entities(self, entities):
        if len(entities) <= 1:
            return entities

        united_result = []
        cur_entity = {key: entities[0][key] for key in ['entity', 'word', 'start', 'end']}
        for entity in entities[1:]:
            if entity['word'].startswith('##'):
                cur_entity['word'] += entity['word'].lstrip('#')
                cur_entity['end'] = entity['end']
            else:
                united_result.append(cur_entity)
                cur_entity = {key: entity[key] for key in ['entity', 'word', 'start', 'end']}
        united_result.append(cur_entity)
        return united_result

    def _convert_entities_to_bio(self, tokens, entities):
        bio_tags = []
        cur_entity_idx = 0
        for token in tokens:
            if (cur_entity_idx < len(entities))\
                    and (token == entities[cur_entity_idx]['word']):
                bio_tags.append(entities[cur_entity_idx]['entity'])
                cur_entity_idx += 1
            else:
                bio_tags.append('O')
        return bio_tags

    def _postprocessing(self, tokens, model_output):
        entities = self._unite_entities(model_output)
        bio_tags = self._convert_entities_to_bio(tokens, entities)
        return bio_tags

    def predict(self, X):
        with torch.no_grad():
            ner_entitites = self.model(X)
            tags = []
            for text, entities in zip(X, ner_entitites):
                tags.append(self._postprocessing(text.split(), entities))
            return tags

wrapped_model = ModelWrapper(model, tokenizer)

In [5]:
def preprocessing_dataset(dataset):
    result = {'X': [], 'y_true': []}
    for sample in dataset:
        result['X'].append(' '.join(sample['tokens']))
        result['y_true'].append([CONLL_NER_TAGS[tag] for tag in sample['ner_tags']])
    return result

In [6]:
# from itertools import islice


# train_data = preprocessing_dataset(conll['train'])
# train_data['y_pred'] = wrapped_model.predict(train_data['X'])

In [7]:

# oos_data = preprocessing_dataset(conll['test'])
# oos_data['y_pred'] = wrapped_model.predict(oos_data['X'])

In [8]:
# import pickle

# with open('./train_data.pkl', 'wb') as f:
#     pickle.dump(train_data, f)
# with open('./oos_data.pkl', 'wb') as f:
#     pickle.dump(oos_data, f)


In [9]:
import pickle

with open('./train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('./oos_data.pkl', 'rb') as f:
    oos_data = pickle.load(f)

In [10]:
all([len(i) == len(j) for i, j in zip(train_data['y_true'], train_data['y_pred'])])


True

In [11]:
all([len(i) == len(j) for i, j in zip(oos_data['y_true'], oos_data['y_pred'])])


True

In [12]:
from sbe_vallib.validation.sampler.ner_sampler import NerSampler

sampler = NerSampler(train=train_data, oos=oos_data)
sampler.set_seed(42, bootstrap=True)

In [13]:
list(set([tag.split('-')[-1] for tag in CONLL_NER_TAGS]) - set(['O']))

['LOC', 'ORG', 'MISC', 'PER']

In [14]:
from sbe_vallib.validation.scorer.ner_scorer import NerScorer
from sbe_vallib.validation.utils.metrics import NER_IOB_METRICS


scorer = NerScorer(metrics=NER_IOB_METRICS, is_calc_ner_metrics=False, tags=['PER', 'MISC', 'LOC', 'ORG'])
scores = scorer.score(oos_data['y_true'], oos_data['y_pred'])

In [17]:
from seqeval.metrics import classification_report

report = classification_report(oos_data['y_true'], oos_data['y_pred'], output_dict=True)
report

{'LOC': {'precision': 0.9363057324840764,
  'recall': 0.7050359712230215,
  'f1-score': 0.8043775649794801,
  'support': 1668},
 'MISC': {'precision': 0.804920913884007,
  'recall': 0.6524216524216524,
  'f1-score': 0.7206923682140047,
  'support': 702},
 'ORG': {'precision': 0.8944262295081967,
  'recall': 0.8211920529801324,
  'f1-score': 0.8562460765850596,
  'support': 1661},
 'PER': {'precision': 0.9319965126416739,
  'recall': 0.6611008039579468,
  'f1-score': 0.7735166425470332,
  'support': 1617},
 'micro avg': {'precision': 0.9043806982432733,
  'recall': 0.7200779036827195,
  'f1-score': 0.8017742730409069,
  'support': 5648},
 'macro avg': {'precision': 0.8919123471294885,
  'recall': 0.7099376201456882,
  'f1-score': 0.7887081630813945,
  'support': 5648},
 'weighted avg': {'precision': 0.9064257739880867,
  'recall': 0.7200779036827195,
  'f1-score': 0.800394646791491,
  'support': 5648}}

In [15]:
sampler.

{'precision': 0.9043806982432733,
 'f1': 0.8017742730409069,
 'recall': 0.7200779036827195}

In [16]:
scorer.ner_metrics(oos_data['y_true'], oos_data['y_pred'])

{'ent_type_f1': 0.8238541153277475,
 'ent_type_f1_by_PER': 0.8030194104960461,
 'ent_type_f1_by_MISC': 0.7612179487179488,
 'ent_type_f1_by_LOC': 0.8062648961525366,
 'ent_type_f1_by_ORG': 0.8829452485840151,
 'ent_type_recall': 0.7399079320113314,
 'ent_type_recall_by_PER': 0.6907854050711194,
 'ent_type_recall_by_MISC': 0.6766381766381766,
 'ent_type_recall_by_LOC': 0.709832134292566,
 'ent_type_recall_by_ORG': 0.8446718844069837,
 'ent_type_precision': 0.9292861907938625,
 'ent_type_precision_by_PER': 0.9587982832618026,
 'ent_type_precision_by_MISC': 0.86996336996337,
 'ent_type_precision_by_LOC': 0.9330181245074862,
 'ent_type_precision_by_ORG': 0.924851680949242,
 'partial_f1': 0.8498767865943814,
 'partial_f1_by_PER': 0.8076923076923076,
 'partial_f1_by_MISC': 0.7908653846153845,
 'partial_f1_by_LOC': 0.8478038815117467,
 'partial_f1_by_ORG': 0.9118942731277533,
 'partial_recall': 0.7632790368271954,
 'partial_recall_by_PER': 0.6948051948051948,
 'partial_recall_by_MISC': 0.7029